# Gender Enrichment for Artist Data

**Gender Assignment Logic:**

1. **Single Artist** → Query MusicBrainz API for gender (`male`, `female`, `non-binary`, `not applicable`, or `None`)

2. **Groups** → Query MusicBrainz for group members and aggregate:
   - All male members → `male`
   - All female members → `female`
   - Mixed, non-binary, or other → `other`

3. **Collaborations/Features** (featuring, &, commas) → Parse individual artists and aggregate:
   - All male artists → `male`
   - All female artists → `female`
   - Mixed, non-binary, or other → `other`

4. **Unknown/Error** → `None`

**Improved Matching Logic:**

- Uses **fuzzy name matching** with configurable similarity threshold (default: 0.85)
- Searches multiple MusicBrainz results (up to 5) and picks the best match
- Normalizes names (lowercase, removes punctuation) before comparison
- Only accepts matches above similarity threshold to avoid false positives
- Debug mode available to inspect matching process

**Note:** Due to MusicBrainz API rate limits (1 request/second), this process may take significant time for unique artists.

## 1. Setup and Imports

In [ ]:
import pandas as pd
import musicbrainzngs
import time
import os
from pyprojroot.here import here
import data_preprocessing

# Configure pandas
pd.options.mode.copy_on_write = True

# Configure MusicBrainz API
musicbrainzngs.set_useragent(
    "ditw-2025-exam",
    "0.1",
    "https://github.com/rakulmaria/ditw-2025-exam"  
)

# Set rate limit to comply with MusicBrainz requirements (1 request per second)
musicbrainzngs.set_rate_limit(limit_or_interval=1.0)

print("Setup complete!")

Setup complete!


## 2. Loadign Dataframes


In [20]:
# Load scraped data
p3_scraped = data_preprocessing.load_df_from_channel('p3')
p4_scraped = data_preprocessing.load_df_from_channel('p4')
p6_scraped = data_preprocessing.load_df_from_channel('p6')

# Reform to minimal structure
p3_oct_2025 = data_preprocessing.reform_datasets_to_minimal(p3_scraped)
p4_oct_2025 = data_preprocessing.reform_datasets_to_minimal(p4_scraped)
p6_oct_2025 = data_preprocessing.reform_datasets_to_minimal(p6_scraped)

# Load DR dataset
datafolder = here() / 'data'
dr_df = pd.read_csv('../dataset_from_DR_minimal.csv', parse_dates=['localTime'])

# Split by channel
p3_2024 = dr_df[dr_df['channel'] == 'P3'].reset_index(drop=True)
p4_2024 = dr_df[dr_df['channel'].str.contains('P4')].reset_index(drop=True)
p6_2024 = dr_df[dr_df['channel'] == 'P6'].reset_index(drop=True)

print(f"Loaded dataframes:")
print(f"  P3 Oct 2025: {len(p3_oct_2025)} rows")
print(f"  P4 Oct 2025: {len(p4_oct_2025)} rows")
print(f"  P6 Oct 2025: {len(p6_oct_2025)} rows")
print(f"  P3 2024: {len(p3_2024)} rows")
print(f"  P4 2024: {len(p4_2024)} rows")
print(f"  P6 2024: {len(p6_2024)} rows")

/Users/frederikrothe/Documents/School/CS5/DIW-DR-project/scripts/data_preprocessing.py:300: FutureWarning: In a future version of pandas, parsing datetimes with mixed time zones will raise an error unless `utc=True`. Please specify `utc=True` to opt in to the new behaviour and silence this warning. To create a `Series` with mixed offsets and `object` dtype, please use `apply` and `datetime.datetime.strptime`
  df['localTime'] = pd.to_datetime(df['localTime'], errors='coerce')
/Users/frederikrothe/Documents/School/CS5/DIW-DR-project/scripts/data_preprocessing.py:300: FutureWarning: In a future version of pandas, parsing datetimes with mixed time zones will raise an error unless `utc=True`. Please specify `utc=True` to opt in to the new behaviour and silence this warning. To create a `Series` with mixed offsets and `object` dtype, please use `apply` and `datetime.datetime.strptime`
  df['localTime'] = pd.to_datetime(df['localTime'], errors='coerce')
/Users/frederikrothe/Documents/School/

Loaded dataframes:
  P3 Oct 2025: 12483 rows
  P4 Oct 2025: 113410 rows
  P6 Oct 2025: 11665 rows
  P3 2024: 135510 rows
  P4 2024: 118016 rows
  P6 2024: 124592 rows


## 3. Helper Functions


In [21]:
import re

def parse_artist_string(artist_string):
    """
    Parse an artist string to extract individual artist names.
    
    Handles:
    - "Artist1 featuring Artist2"
    - "Artist1 feat. Artist2"
    - "Artist1 ft. Artist2"
    - "Artist1 & Artist2"
    - "Artist1, Artist2"
    - Combinations of the above
    
    Args:
        artist_string (str): The artist name(s)
    
    Returns:
        list: List of individual artist names, or single item list if no collaboration detected
    """
    if pd.isna(artist_string):
        return []
    
    artist_string = str(artist_string)
    
    # Replace collaboration indicators with a common separator
    separators = [
        (r'\s+featuring\s+', '||', re.IGNORECASE),
        (r'\s+feat\.\s+', '||', re.IGNORECASE),
        (r'\s+ft\.\s+', '||', re.IGNORECASE),
        (r'\s+&\s+', '||', 0),
        (r',\s+', '||', 0)
    ]
    
    for pattern, replacement, flags in separators:
        artist_string = re.sub(pattern, replacement, artist_string, flags=flags)
    
    # Split on the common separator and clean up
    artists = [name.strip() for name in artist_string.split('||') if name.strip()]
    
    return artists if artists else [artist_string]


def is_multiple_artists(artist_string):
    """
    Detect if the artist string contains multiple artists.
    
    Returns True if:
    - Contains 'featuring', 'feat.', 'ft.'
    - Contains ' & ' (ampersand with spaces)
    - Contains ', ' (comma with space)
    
    Args:
        artist_string (str): The artist name(s)
    
    Returns:
        bool: True if multiple artists detected
    """
    if pd.isna(artist_string):
        return False
    
    artist_string = str(artist_string).lower()
    
    # Check for collaboration indicators
    indicators = ['featuring', 'feat.', 'ft.', ' & ', ', ']
    
    return any(indicator in artist_string for indicator in indicators)

In [22]:
def load_gender_cache(csv_path):
    """
    Load existing artist→gender mappings from CSV file.
    
    Args:
        csv_path (Path): Path to cache CSV file
    
    Returns:
        dict: Dictionary mapping artist names to genders
    """
    if not os.path.exists(csv_path):
        print(f"No existing cache found at {csv_path}")
        return {}
    
    try:
        cache_df = pd.read_csv(csv_path)
        cache_dict = dict(zip(cache_df['artist'], cache_df['gender']))
        print(f"Loaded {len(cache_dict)} cached artist→gender mappings")
        return cache_dict
    except Exception as e:
        print(f"Error loading cache: {e}")
        return {}


def save_gender_cache(cache_dict, csv_path):
    """
    Save artist→gender mappings to CSV file.
    
    Args:
        cache_dict (dict): Dictionary mapping artist names to genders
        csv_path (Path): Path to save cache CSV file
    """
    try:
        cache_df = pd.DataFrame([
            {'artist': artist, 'gender': gender}
            for artist, gender in cache_dict.items()
        ])
        cache_df.to_csv(csv_path, index=False)
        print(f"Saved {len(cache_dict)} mappings to {csv_path}")
    except Exception as e:
        print(f"Error saving cache: {e}")

In [23]:
from difflib import SequenceMatcher
import time

def normalize_artist_name(name):
    """
    Normalize artist name for comparison by removing special characters,
    converting to lowercase, and stripping whitespace.
    
    Args:
        name (str): Artist name
    
    Returns:
        str: Normalized name
    """
    if not name:
        return ""
    # Convert to lowercase, remove extra spaces
    name = str(name).lower().strip()
    # Remove common punctuation that might differ
    for char in ['.', ',', '!', '?', ';', ':']:
        name = name.replace(char, '')
    return ' '.join(name.split())  # Normalize whitespace


def name_similarity(name1, name2):
    """
    Calculate similarity ratio between two artist names.
    
    Args:
        name1 (str): First name
        name2 (str): Second name
    
    Returns:
        float: Similarity ratio between 0 and 1
    """
    norm1 = normalize_artist_name(name1)
    norm2 = normalize_artist_name(name2)
    return SequenceMatcher(None, norm1, norm2).ratio()


def aggregate_genders(genders):
    """
    Aggregate a list of genders according to the rules:
    - All male → 'male'
    - All female → 'female'
    - Any mix, non-binary, or other → 'other'
    
    Args:
        genders (list): List of gender strings
    
    Returns:
        str: Aggregated gender
    """
    # Filter out None values
    valid_genders = [g for g in genders if g is not None]
    
    if not valid_genders:
        return 'other'
    
    # Get unique genders
    unique_genders = set(valid_genders)
    
    # All male → 'male'
    if unique_genders == {'male'}:
        return 'male'
    
    # All female → 'female'
    if unique_genders == {'female'}:
        return 'female'
    
    # Everything else → 'other'
    return 'other'


def get_single_artist_gender(artist_name, cache, similarity_threshold=0.85, debug=False, max_retries=3):
    """
    Query MusicBrainz for a single artist's gender (no features/collaborations).
    
    For groups, checks member genders and aggregates them.
    Now includes name matching to avoid incorrect results and retry logic for network errors.
    
    Args:
        artist_name (str): Name of a single artist
        cache (dict): Cache dictionary
        similarity_threshold (float): Minimum similarity ratio for name matching (0-1)
        debug (bool): If True, print debug information
        max_retries (int): Maximum number of retry attempts for network errors
    
    Returns:
        str or None: 'male', 'female', 'other', or None if not found
    """
    retry_count = 0
    last_error = None
    
    while retry_count <= max_retries:
        try:
            # Search for multiple results to find best match
            result = musicbrainzngs.search_artists(artist=artist_name, limit=5)
            
            if not result['artist-list']:
                if debug:
                    print(f"  No results found for '{artist_name}'")
                return None
            
            # Find the best matching artist
            best_match = None
            best_similarity = 0
            
            for artist in result['artist-list']:
                artist_mb_name = artist.get('name', '')
                similarity = name_similarity(artist_name, artist_mb_name)
                
                if debug:
                    print(f"  Candidate: '{artist_mb_name}' (similarity: {similarity:.2f})")
                
                if similarity > best_similarity:
                    best_similarity = similarity
                    best_match = artist
            
            # Check if best match meets threshold
            if best_similarity < similarity_threshold:
                if debug:
                    print(f"  ❌ Best match '{best_match.get('name', '')}' below threshold "
                          f"({best_similarity:.2f} < {similarity_threshold})")
                return None
            
            artist = best_match
            if debug:
                print(f"  ✓ Matched: '{artist.get('name', '')}' (similarity: {best_similarity:.2f})")
            
            artist_type = artist.get('type', '').lower()
            
            # If it's a group, check member genders
            if artist_type == 'group':
                artist_id = artist.get('id')
                if artist_id:
                    return get_group_gender(artist_id, cache, max_retries=max_retries)
                else:
                    return 'other'
            else:
                # Individual artist - return their gender
                gender = artist.get('gender', None)
                if gender:
                    gender = gender.lower()
                if debug:
                    print(f"  Gender: {gender}")
                return gender
                
        except musicbrainzngs.NetworkError as e:
            last_error = e
            retry_count += 1
            if retry_count <= max_retries:
                wait_time = 2 ** retry_count  # Exponential backoff: 2s, 4s, 8s
                print(f"  ⚠️ Network error for '{artist_name}': {e}")
                print(f"     Retrying in {wait_time}s... (attempt {retry_count}/{max_retries})")
                time.sleep(wait_time)
            else:
                print(f"  ❌ Failed after {max_retries} retries for '{artist_name}': {e}")
                return None
                
        except Exception as e:
            print(f"Error querying single artist '{artist_name}': {e}")
            return None
    
    # If we've exhausted retries
    print(f"❌ Max retries exceeded for '{artist_name}': {last_error}")
    return None


def get_group_gender(artist_id, cache, max_retries=3):
    """
    Query MusicBrainz for group members and determine aggregate gender.
    Now includes retry logic for network errors.
    
    Args:
        artist_id (str): MusicBrainz artist ID
        cache (dict): Cache dictionary
        max_retries (int): Maximum number of retry attempts for network errors
    
    Returns:
        str: 'male' if all members are male, 'female' if all female, 'other' otherwise
    """
    retry_count = 0
    last_error = None
    
    while retry_count <= max_retries:
        try:
            # Get artist details including members
            artist_details = musicbrainzngs.get_artist_by_id(
                artist_id,
                includes=['artist-rels']
            )
            
            # Extract member relationships
            artist_info = artist_details.get('artist', {})
            relations = artist_info.get('artist-relation-list', [])
            
            # Get members' genders
            member_genders = []
            
            for relation in relations:
                if relation.get('type') in ['member of band', 'member']:
                    member = relation.get('artist', {})
                    member_gender = member.get('gender', '').lower()
                    
                    if member_gender:
                        member_genders.append(member_gender)
            
            # If no members found, return 'other'
            if not member_genders:
                return 'other'
            
            # Aggregate member genders
            return aggregate_genders(member_genders)
            
        except musicbrainzngs.NetworkError as e:
            last_error = e
            retry_count += 1
            if retry_count <= max_retries:
                wait_time = 2 ** retry_count  # Exponential backoff
                print(f"  ⚠️ Network error getting group members for ID {artist_id}: {e}")
                print(f"     Retrying in {wait_time}s... (attempt {retry_count}/{max_retries})")
                time.sleep(wait_time)
            else:
                print(f"  ❌ Failed after {max_retries} retries for group ID {artist_id}")
                return 'other'
                
        except Exception as e:
            print(f"Error getting group members for ID {artist_id}: {e}")
            return 'other'
    
    # If we've exhausted retries
    print(f"❌ Max retries exceeded for group ID {artist_id}")
    return 'other'


def query_musicbrainz_gender(artist_name, cache, similarity_threshold=0.85, debug=False, max_retries=3):
    """
    Query MusicBrainz API for artist gender with caching.
    
    Handles:
    - Single artists: Returns their gender
    - Groups: Aggregates member genders
    - Collaborations/Features: Parses artists and aggregates their genders
    - Network errors: Retries with exponential backoff
    
    Aggregation rules:
    - All male → 'male'
    - All female → 'female'
    - Mixed/non-binary/other → 'other'
    
    Args:
        artist_name (str): Name of the artist(s)
        cache (dict): Cache dictionary to check/update
        similarity_threshold (float): Minimum similarity ratio for name matching (0-1)
        debug (bool): If True, print debug information
        max_retries (int): Maximum number of retry attempts for network errors
    
    Returns:
        str or None: 'male', 'female', 'other', or None if not found
    """
    # Check cache first
    if artist_name in cache:
        return cache[artist_name]
    
    # Check if this is a collaboration/feature
    if is_multiple_artists(artist_name):
        # Parse individual artists
        individual_artists = parse_artist_string(artist_name)
        
        # Query each individual artist
        individual_genders = []
        for artist in individual_artists:
            # Check if individual artist is in cache
            if artist in cache:
                gender = cache[artist]
            else:
                # Query and cache individual artist
                gender = get_single_artist_gender(artist, cache, similarity_threshold, debug, max_retries)
                cache[artist] = gender
            
            individual_genders.append(gender)
        
        # Aggregate the genders
        aggregated_gender = aggregate_genders(individual_genders)
        cache[artist_name] = aggregated_gender
        return aggregated_gender
    
    else:
        # Single artist (or group)
        gender = get_single_artist_gender(artist_name, cache, similarity_threshold, debug, max_retries)
        cache[artist_name] = gender
        return gender

In [24]:
# Test with Svea S
print("=" * 60)
print("Testing: Svea S")
print("=" * 60)

test_cache = {}
result = query_musicbrainz_gender("Svea S", test_cache, similarity_threshold=0.85, debug=True)

print(f"\n{'='*60}")
print(f"Result: {result}")
print(f"{'='*60}\n")

# Also test with lower threshold to see what happens
print("\nTesting with lower threshold (0.70):")
print("=" * 60)
test_cache2 = {}
result2 = query_musicbrainz_gender("Svea S", test_cache2, similarity_threshold=0.70, debug=True)
print(f"\n{'='*60}")
print(f"Result with threshold 0.70: {result2}")
print(f"{'='*60}")

INFO:musicbrainzngs:in <ws2:artist>, uncaught attribute type-id
INFO:musicbrainzngs:in <ws2:area>, uncaught attribute type-id
INFO:musicbrainzngs:in <ws2:begin-area>, uncaught attribute type-id
INFO:musicbrainzngs:in <ws2:end-area>, uncaught attribute type-id
INFO:musicbrainzngs:in <ws2:alias>, uncaught attribute type-id
INFO:musicbrainzngs:in <ws2:artist>, uncaught attribute type-id
INFO:musicbrainzngs:in <ws2:area>, uncaught attribute type-id
INFO:musicbrainzngs:in <ws2:artist>, uncaught attribute type-id
INFO:musicbrainzngs:in <ws2:area>, uncaught attribute type-id
INFO:musicbrainzngs:in <ws2:begin-area>, uncaught attribute type-id
INFO:musicbrainzngs:in <ws2:end-area>, uncaught attribute type-id
INFO:musicbrainzngs:in <ws2:alias>, uncaught attribute type-id
INFO:musicbrainzngs:in <ws2:artist>, uncaught attribute type-id
INFO:musicbrainzngs:in <ws2:area>, uncaught attribute type-id
INFO:musicbrainzngs:in <ws2:artist>, uncaught attribute type-id
INFO:musicbrainzngs:in <ws2:area>, unc

Testing: Svea S
  Candidate: 'C. S. Lewis' (similarity: 0.27)
  Candidate: 'Svea S' (similarity: 1.00)
  Candidate: 'S. P. Balasubrahmanyam' (similarity: 0.23)
  Candidate: 'Pluton Svea' (similarity: 0.47)
  Candidate: 'Robin S' (similarity: 0.31)
  ✓ Matched: 'Svea S' (similarity: 1.00)
  Gender: female

Result: female


Testing with lower threshold (0.70):


INFO:musicbrainzngs:in <ws2:artist>, uncaught attribute type-id
INFO:musicbrainzngs:in <ws2:area>, uncaught attribute type-id
INFO:musicbrainzngs:in <ws2:begin-area>, uncaught attribute type-id
INFO:musicbrainzngs:in <ws2:end-area>, uncaught attribute type-id
INFO:musicbrainzngs:in <ws2:alias>, uncaught attribute type-id
INFO:musicbrainzngs:in <ws2:artist>, uncaught attribute type-id
INFO:musicbrainzngs:in <ws2:area>, uncaught attribute type-id
INFO:musicbrainzngs:in <ws2:artist>, uncaught attribute type-id
INFO:musicbrainzngs:in <ws2:area>, uncaught attribute type-id
INFO:musicbrainzngs:in <ws2:begin-area>, uncaught attribute type-id
INFO:musicbrainzngs:in <ws2:end-area>, uncaught attribute type-id
INFO:musicbrainzngs:in <ws2:alias>, uncaught attribute type-id
INFO:musicbrainzngs:in <ws2:artist>, uncaught attribute type-id
INFO:musicbrainzngs:in <ws2:area>, uncaught attribute type-id
INFO:musicbrainzngs:in <ws2:artist>, uncaught attribute type-id
INFO:musicbrainzngs:in <ws2:area>, unc

  Candidate: 'C. S. Lewis' (similarity: 0.27)
  Candidate: 'Svea S' (similarity: 1.00)
  Candidate: 'S. P. Balasubrahmanyam' (similarity: 0.23)
  Candidate: 'Pluton Svea' (similarity: 0.47)
  Candidate: 'Robin S' (similarity: 0.31)
  ✓ Matched: 'Svea S' (similarity: 1.00)
  Gender: female

Result with threshold 0.70: female


In [ ]:
# Test multiple artists
test_artists = [
    "Svea S",           # Danish female artist
    "Harry Styles",     # Well-known male artist
    "Billie Eilish",    # Well-known female artist
    "The Minds Of 99",  # Danish band
]

print("Testing multiple artists with improved matching:")
print("=" * 60)

test_batch_cache = {}
results = []

for artist in test_artists:
    print(f"\n{'='*60}")
    print(f"Testing: {artist}")
    print(f"{'='*60}")
    
    gender = query_musicbrainz_gender(artist, test_batch_cache, similarity_threshold=0.85, debug=True)
    results.append((artist, gender))
    print(f"Result: {gender}\n")
    
    # Add small delay to respect rate limit
    time.sleep(1.1)

print("\n" + "=" * 60)
print("SUMMARY")
print("=" * 60)
for artist, gender in results:
    print(f"{artist:30} → {gender}")

## 4. Query Process

Extract unique artists and query MusicBrainz for gender information.


In [26]:
# Combine all dataframes to get unique artists
all_dfs = [p3_oct_2025, p4_oct_2025, p6_oct_2025, p3_2024, p4_2024, p6_2024]
all_artists = pd.concat([df['artistString'] for df in all_dfs]).dropna().unique()

print(f"Total unique artists to process: {len(all_artists)}")
print(f"\nEstimated time (at 1 req/sec): ~{len(all_artists) / 60:.1f} minutes")
print(f"\nFirst 10 artists:")
for i, artist in enumerate(all_artists[:10], 1):
    print(f"  {i}. {artist}")

Total unique artists to process: 11269

Estimated time (at 1 req/sec): ~187.8 minutes

First 10 artists:
  1. Lola Young
  2. Milky Chance
  3. Pil
  4. Hozier
  5. Ganger
  6. Tobias Rahim, Mille
  7. Justin Bieber
  8. Libianca
  9. Svea S
  10. Zar Paulo


In [ ]:
# Load existing cache
cache_path = here() / 'data' / 'artist_gender_cache.csv'
gender_cache = load_gender_cache(cache_path)

# Filter cache: Keep only 'male' and 'female', re-query everything else
print(f"Original cache size: {len(gender_cache)}")
print("\nOriginal distribution:")
for value in ['male', 'female', 'other', 'non-binary', 'not applicable', None]:
    count = sum(1 for v in gender_cache.values() if v == value)
    if count > 0:
        print(f"  - {value}: {count}")

# Create filtered cache (keep only 'male' and 'female')
filtered_cache = {k: v for k, v in gender_cache.items() if v in ['male', 'female']}
print(f"\nFiltered cache size (keeping only 'male' and 'female'): {len(filtered_cache)}")
print(f"  - Will re-query: {len(gender_cache) - len(filtered_cache)} artists")

# Use filtered cache
gender_cache = filtered_cache

# Count how many we still need to query
artists_to_query = [a for a in all_artists if a not in gender_cache]
print(f"\nArtists to query: {len(artists_to_query)}")
print(f"Estimated time: ~{len(artists_to_query) / 60:.1f} minutes")
print(f"Note: Group member lookups will add additional time\n")

# Configuration for improved matching
SIMILARITY_THRESHOLD = 0.85  # Minimum similarity for name matching (0-1)
DEBUG_MODE = False           # Set to True to see matching details

print(f"Using similarity threshold: {SIMILARITY_THRESHOLD}")
print(f"Debug mode: {DEBUG_MODE}\n")

# Query MusicBrainz for each artist not in cache
save_interval = 50  # Save cache every 50 queries

for i, artist in enumerate(artists_to_query, 1):
    gender = query_musicbrainz_gender(
        artist, 
        gender_cache, 
        similarity_threshold=SIMILARITY_THRESHOLD,
        debug=DEBUG_MODE
    )
    
    # Progress indicator
    if i % 10 == 0:
        print(f"Progress: {i}/{len(artists_to_query)} ({i/len(artists_to_query)*100:.1f}%) - Last: {artist[:50]} → {gender}")
    
    # Save cache periodically
    if i % save_interval == 0:
        save_gender_cache(gender_cache, cache_path)
        print(f"  💾 Cache saved at {i} queries")

# Final save
save_gender_cache(gender_cache, cache_path)
print(f"\n✓ Enrichment complete! Total artists in cache: {len(gender_cache)}")

In [34]:
# Show statistics
cache_df = pd.DataFrame([
    {'artist': artist, 'gender': gender}
    for artist, gender in gender_cache.items()
])

print("\nGender Distribution in Cache:")
print(cache_df['gender'].value_counts(dropna=False))
print(f"\nTotal: {len(cache_df)} artists")


Gender Distribution in Cache:
gender
other             5338
male              4515
None              2274
female            1997
non-binary          24
not applicable       2
Name: count, dtype: int64

Total: 14150 artists


## 5. Apply Gender to Dataframes

Map the gender information back to all dataframes.

In [35]:
def add_gender_column(df, gender_cache):
    """
    Add gender column to dataframe using the cache.
    
    Args:
        df (DataFrame): Dataframe with artistString column
        gender_cache (dict): Artist→gender mapping
    
    Returns:
        DataFrame: Dataframe with new gender column
    """
    df = df.copy()
    df['gender'] = df['artistString'].map(gender_cache)
    return df


# Apply to all dataframes
p3_oct_2025 = add_gender_column(p3_oct_2025, gender_cache)
p4_oct_2025 = add_gender_column(p4_oct_2025, gender_cache)
p6_oct_2025 = add_gender_column(p6_oct_2025, gender_cache)
p3_2024 = add_gender_column(p3_2024, gender_cache)
p4_2024 = add_gender_column(p4_2024, gender_cache)
p6_2024 = add_gender_column(p6_2024, gender_cache)

print("✓ Gender column added to all dataframes!")

✓ Gender column added to all dataframes!


In [36]:
# Show sample results
print("Sample from P3 Oct 2025:")
print(p3_oct_2025[['artistString', 'gender']].head(10))

print("\n" + "="*50 + "\n")

# Gender distribution per dataframe
dataframes = [
    ('P3 Oct 2025', p3_oct_2025),
    ('P4 Oct 2025', p4_oct_2025),
    ('P6 Oct 2025', p6_oct_2025),
    ('P3 2024', p3_2024),
    ('P4 2024', p4_2024),
    ('P6 2024', p6_2024)
]

for name, df in dataframes:
    print(f"\n{name} - Gender Distribution:")
    print(df['gender'].value_counts(dropna=False))
    print(f"  Total rows: {len(df)}")

Sample from P3 Oct 2025:
          artistString  gender
0           Lola Young  female
1         Milky Chance   other
2                  Pil  female
3               Hozier    male
4               Ganger   other
5  Tobias Rahim, Mille    male
6        Justin Bieber    male
7             Libianca  female
8               Svea S  female
9            Zar Paulo   other



P3 Oct 2025 - Gender Distribution:
gender
male          4203
female        3985
other         3338
None           837
non-binary     120
Name: count, dtype: int64
  Total rows: 12483

P4 Oct 2025 - Gender Distribution:
gender
other         37046
female        33769
male          32430
None           9675
non-binary      488
NaN               2
Name: count, dtype: int64
  Total rows: 113410

P6 Oct 2025 - Gender Distribution:
gender
other             7734
female            1448
male              1367
None              1109
non-binary           3
not applicable       2
NaN                  2
Name: count, dtype: int64
  Total 

## 6. Save Enriched Dataframes (Optional)

Save the enriched dataframes to CSV files if needed.

In [ ]:
output_folder = here() / 'data'
output_folder.mkdir(exist_ok=True)

p3_oct_2025.to_csv(output_folder / 'p3_oct_2025.csv', index=False)
p4_oct_2025.to_csv(output_folder / 'p4_oct_2025.csv', index=False)
p6_oct_2025.to_csv(output_folder / 'p6_oct_2025.csv', index=False)
p3_2024.to_csv(output_folder / 'p3_2024.csv', index=False)
p4_2024.to_csv(output_folder / 'p4_2024.csv', index=False)
p6_2024.to_csv(output_folder / 'p6_2024.csv', index=False)

print(f"Saved enriched dataframes to {output_folder}")

print("Ready to save! Uncomment the code above to save enriched dataframes.")

Saved enriched dataframes to /Users/frederikrothe/Documents/School/CS5/DIW-DR-project/data/enriched_also_groups
Ready to save! Uncomment the code above to save enriched dataframes.
